### ***Multilayer Perceptron and Backpropagation***

In [185]:
''' Import all important library '''
import torch
import torch.nn as nn
import torch.optim as gredient
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [186]:
Device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device Name is {Device}')

Device Name is cpu


### ***Data Pipeline***

In [187]:
Transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.1307),
        std=(0.3181)
    )
])
'''
Compose = all number convert between (0 - 1)[pixel_num / 255]
standard division
'''

'\nCompose = all number convert between (0 - 1)[pixel_num / 255]\nstandard division\n'

In [188]:
Train_Dataset = datasets.MNIST(
    root='/Users/mahadiur/Desktop/Deep Learning Forward and Backpropagation/Data',
    train=True,
    download=True,
    transform=Transformation
)

Test_Dataset = datasets.MNIST(
    root='/Users/mahadiur/Desktop/Deep Learning Forward and Backpropagation/Data',
    train=False,
    download=True,
    transform=Transformation
)

In [189]:
print(type(Train_Dataset))

<class 'torchvision.datasets.mnist.MNIST'>


### ***Dataloader***

In [190]:
Train_Dataloader = DataLoader(
    Train_Dataset,
    batch_size=32,
    shuffle=True
)

Test_Dataloader = DataLoader(
    Test_Dataset,
    batch_size=32,
    shuffle=False
)

### ***Perceptron***

In [191]:
class Perceptron(nn.Module):
    def __init__(self, input_size):
        super(Perceptron, self).__init__()
        self.weights = nn.Parameter(torch.randn(input_size))
        self.bias = nn.Parameter(torch.randn(1))
    
    def forward(self, x):
        sumation = x @ self.weights + self.bias
        return sumation

In [192]:
''' Testing '''
input_size = 28 * 28
shuffle = torch.randn(1, input_size)

perceptron = Perceptron(
    input_size=input_size
)

output = perceptron(shuffle)
print(output)

tensor([-59.9092], grad_fn=<AddBackward0>)


In [193]:
'''
Some import term for image classification:
- Batch Size (8, 32, 64, 128)
- epoch
- Step 
MNIST Dataset :
- Grayscale image 
- pixel(28 * 28)
- 60000 images in MNIST dataset
'''

'\nSome import term for image classification:\n- Batch Size (8, 32, 64, 128)\n- epoch\n- Step \nMNIST Dataset :\n- Grayscale image \n- pixel(28 * 28)\n- 60000 images in MNIST dataset\n'

### ***Activation Function***

In [194]:
'''
Activation Function: RELU (Rectified Linear Unit)
Function: max(0, z)  
'''
class RELU(nn.Module):
    def __init__(self):
        super(RELU, self).__init__()
    def forward(self, x):
        return torch.maximum(torch.tensor(0.0), x)

In [195]:
relu = RELU()
output_relu = relu(output)
print(output_relu)

tensor([0.], grad_fn=<MaximumBackward0>)


### ***Perceptron Hidden Layer***

In [197]:
''' Perceptron Hidden Layer '''
class Linear(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.perceptron = nn.ModuleList([
            Perceptron(input_size) for _ in range(output_size)
        ])
    def forward(self, x):
        outputs = [
            perceptron(x) for perceptron in self.perceptron
        ]
        final_output =torch.stack(outputs, dim=1)
        return final_output


# Testing
linear = Linear(input_size,10) 
output = linear(shuffle)
print(output)
print(output.shape)

tensor([[-66.4359,  26.2814, -16.4357, -56.2052,  -8.2585,  -7.4041,   3.6872,
          14.4406, -13.0250,  12.4655]], grad_fn=<StackBackward0>)
torch.Size([1, 10])


### ***Digit Classifiar betweet (1, 10)***

In [203]:
class Digitclassifiar(nn.Module):
    def __init__(self, input_size=28 * 28, output_size=10):
        super().__init__()
        self.layer1 = Linear(input_size, 256 )
        self.layer2 = Linear(256, 128)
        self.layer3 = Linear(128, 64)
        self.layer4 = Linear(64, output_size)
        self.relu = RELU()
    
    def forward(self, x):
        x = x.view(-1, input_size) # [1 ... 784]
        x = self.layer1(x) # [1 ... 256]
        x = self.relu(x) # activation function
        x = self.layer2(x) # [1...128]
        x = self.relu(x)
        x = self.layer3(x) # [1...64]
        x = self.relu(x)
        x = self.layer4(x) # [1...10]
        return x

In [204]:
Model = Digitclassifiar(input_size=28 * 28, output_size=10).to(Device)
output = Model(shuffle)
print(output.shape)
print(output)

torch.Size([1, 10])
tensor([[-15583.4268,  19430.7344, -21702.2656, -12942.6582,   -311.5664,
         -13891.0508,  -3756.3945,  -7307.3623,  24243.7969, -13277.1035]],
       grad_fn=<StackBackward0>)
